In [25]:
from pyspark.sql import SparkSession, SQLContext

spark = SparkSession \
    .builder \
    .appName("shud") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
    
sqlContext = SQLContext(spark)    
initList = [("", "")]


df = spark.createDataFrame(initList, schema=["url", "parsed"])
df = df.union(spark.createDataFrame([("https://amazon.com/xx", "false")]))
df = df.union(spark.createDataFrame([("https://amazon.com/xx", "false")]))
df = df.union(spark.createDataFrame([("https://amazon.com/xy", "false")]))
df = df.dropDuplicates()

#df.show()

sqlContext.registerDataFrameAsTable(df, "UrlList")
#tt = "table1" in sqlContext.tableNames()
#df2 = sqlContext.sql("SELECT url, parsed from UrlList where parsed = 'false'")
#df2.collect()

l = ['x','y']
df = spark.createDataFrame((l, 'false'), schema=["url", "parsed"])
df.show() 


https://amazon.com/xx
Réussi
https://patate.com/xx
